In [ ]:
# imports and settings

import os
import time
import pickle
import warnings
import pandas as pd
import networkx as nx
from networkx.algorithms.coloring import greedy_color
import matplotlib.pyplot as plt
from copy import deepcopy

import numpy as np
from numpy import linalg as LA
from numpy import histogram2d

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt, welch, get_window
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile
from scipy.stats import wasserstein_distance_nd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, linear_kernel

import utils as ut
%load_ext autoreload
%autoreload 2

# do not show warnings
warnings.filterwarnings("ignore")

print("Imports complete.")

In [ ]:
src = "../data/לאיתמר_באהבה"
all_data = []
all_fs = []
names = []
min_distance = [823, 653, 482, 312, 142, 10]
for f in sorted(os.listdir(src)):
    if f.endswith(".wav"):
        fs, data = wavfile.read(os.path.join(src, f))
        all_fs.append(fs)
        all_data.append(data)
        names.append(f)
        print(f"Loaded {f} with sampling rate {fs} Hz and shape {data.shape}.")
        

In [ ]:
# # only use the first and last files for testing
# all_data = [all_data[0], all_data[-1]]
# all_fs = [all_fs[0], all_fs[-1]]
# names = [names[0], names[-1]]

In [ ]:
fs = all_fs[0]
nperseg = 32000
overlap = 0.5
window = 'hanning'
dc = 100
crop_freq = 32000
norm_size = 5
s2g_quantization_levels = 10
s2g_mode = 'laplacian'
welch_threshold = 0.034
s2g_threshold = 0.24

fig = make_subplots(rows=3, 
                    cols=len(all_data),
                    vertical_spacing=0.05, 
                    horizontal_spacing=0.02,
                    row_heights=[0.8, 0.1, 0.1],
                    shared_xaxes=True,
                    row_titles=["Spectrogram", "Welch-CFAR", "S2G K"], 
                    column_titles=[f"min distance: {d}m" for d in min_distance])


welch_detector = ut.WelchDetector(all_fs[0], nperseg=nperseg, overlap=overlap, window=window, dc=dc, crop_freq=crop_freq, norm_size=norm_size)
s2g_detector = ut.S2GDetector(all_fs[0], nperseg=nperseg, overlap=overlap, window=window, dc=dc, crop_freq=crop_freq, mode=s2g_mode, quantization_levels=s2g_quantization_levels)

for i in range(len(all_data)):
    F, T, Sxx, phasogram = ut.calc_spectrogram(all_data[i], all_fs[i], nperseg=nperseg, percent_overlap=overlap, window=window, remove_dc=dc, crop_freq=crop_freq)
    _, pxx_detections, (pxx, pxx_f) = welch_detector.detect(all_data[i], welch_threshold)
s    _, K_detections, (K, K_f) = s2g_detector.detect(all_data[i], s2g_threshold)

    fig.add_trace(go.Heatmap(z=Sxx.T, x=F, y=T, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=i+1)
    fig.add_trace(go.Scatter(x=pxx_f, y=pxx, mode='lines', line=dict(color='blue'), showlegend=False), row=2, col=i+1)
    fig.add_trace(go.Scatter(x=pxx_f[pxx_detections], y=pxx[pxx_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=2, col=i+1)
    fig.add_trace(go.Scatter(x=K_f, y=K, mode='lines', line=dict(color='purple'), showlegend=False), row=3, col=i+1)
    fig.add_trace(go.Scatter(x=K_f[K_detections], y=K[K_detections], mode='markers', marker=dict(color='red', size=10), showlegend=False), row=3, col=i+1)

fig.update_layout(height=900, width=800*len(all_data), title_text="Spectrogram and Detection Metrics for All Recordings")
fig.show()